<a href="https://colab.research.google.com/github/youssefbourais0310/Machine_learning_regrission/blob/main/car_sales_missing_data_Sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fill missing values with scikit-learn


In [1]:
#standard imports
import numpy as np 
import pandas as pd 

In [2]:
#import car sales missing data  
car_sales_missing = pd.read_csv("/content/drive/MyDrive/car-sales-extended-missing-data.csv")     
car_sales_missing.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0


In [3]:
car_sales_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [4]:
#Drop the rows with no labels
car_sales_missing.dropna(subset=["Price"] , inplace=True)
car_sales_missing.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [5]:
#split X and y 
X = car_sales_missing.drop("Price", axis=1)
y = car_sales_missing["Price"]

In [6]:
X.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
dtype: int64

In [7]:
# Fill missing values with Scikit-Learn
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Fill categorical values with 'missing' & numerical values with mean
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")
door_imputer = SimpleImputer(strategy="constant", fill_value=4)
#replace missing values using the mean along each column. Can only be used with numeric data.
num_imputer = SimpleImputer(strategy="mean")

# Define columns
cat_features = ["Make", "Colour"]
door_feature = ["Doors"]
num_features = ["Odometer (KM)"] 

# Create an imputer (something that fills missing data)
imputer = ColumnTransformer([
    ("cat_imputer", cat_imputer, cat_features),
    ("door_imputer", door_imputer, door_feature),
    ("num_imputer", num_imputer, num_features)
])

#transform the data 
filled_X = imputer.fit_transform(X)
filled_X

# Fill train and test values separately
#filled_X_train = imputer.fit_transform(X_train) # fit_transform imputes the missing values from the training set and fills them simultaneously
#filled_X_test = imputer.transform(X_test) # tranform takes the imputing missing values from the training set and fills the test set with them

# Check filled X_train
#filled_X_train

array([['Honda', 'White', 4.0, 35431.0],
       ['BMW', 'Blue', 5.0, 192714.0],
       ['Honda', 'White', 4.0, 84714.0],
       ...,
       ['Nissan', 'Blue', 4.0, 66604.0],
       ['Honda', 'White', 4.0, 215883.0],
       ['Toyota', 'Blue', 4.0, 248360.0]], dtype=object)

In [12]:
car_sales_filled=pd.DataFrame(filled_X,columns=["Make","colour", "Doors","Odometer (KM)"])
car_sales_filled.head()

,Make,colour,Doors,Odometer (KM)
0,Honda,White,4,35431
1,BMW,Blue,5,192714
2,Honda,White,4,84714
3,Toyota,White,4,154365
4,Nissan,Blue,3,181577


In [9]:
car_sales_filled.isna().sum()

Make             0
colour           0
Doors            0
Odometer (KM)    0
dtype: int64

In [10]:
#lets try and convert our data to numbers
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = ["Make" , "colour" , "Doors"]
one_hot = OneHotEncoder()
transformer = ColumnTransformer( [("one_hot", one_hot , categorical_features )], remainder = "passthrough")

transformed_X = transformer.fit_transform(car_sales_filled)
transformed_X

<950x15 sparse matrix of type '<class 'numpy.float64'>'
	with 3800 stored elements in Compressed Sparse Row format>

In [11]:
#Now we ve got our data as numbers and filled (no missing values)
#lets fit the model 
np.random.seed(42)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Split data into train and test
X_train,X_test , y_train , y_test = train_test_split(transformed_X,y,test_size=0.2)

model = RandomForestRegressor()
model.fit(X_train, y_train) 
model.score(X_test,y_test)

0.21990196728583944